In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from apps.environment_simulator.models import SimulatedStockBuffer
from apps.environment_simulator.sevice_layer.stock_simulator import StockSimulator
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf
from django.db.models import Q

In [2]:
ognl = 102.76960814733987
abc = 6.737933572495763
perc = 0.06556348412689915
covar = +0.19172075941288885

fnl = covar*perc
print(fnl)

0.012569880966563989


In [10]:
np.random.seed(3)
for i in range(200):
    starting_latest_time_step = SimulatedStockBuffer.objects.first().captured_at
    curr_month = starting_latest_time_step.month
    # starting_latest_time_step = pytz.utc.localize(datetime.strptime(str(starting_latest_time_step), '%Y-%m-%d %H:%M:%S'))
    starting_next_time_step: datetime = starting_latest_time_step + relativedelta(hours= 2, minutes=30)
    next_month = starting_next_time_step.month
    latest_time_step = starting_latest_time_step
    next_time_step = starting_next_time_step
    stck_smltr = StockSimulator(latest_time_step, next_time_step, is_coupled=True)
    if next_month > curr_month:
        stck_smltr.reset_sentiments()
    stck_smltr.vary_stock_prices()

In [4]:
set = 10.40094383806864
x = 10.40094383806864 + set*(-0.0020001797619766054 - 0.010002471279435696 + 0.010960868361941743 + 0.0038893709977549295 + 0.050702439575785074 - 0.03415644894816287)
print(x)
print(28.5/28.583523116499876)

10.602655363504164
0.9970779278621654


In [12]:
cf.go_offline()
cf.set_config_file(world_readable=True, theme="white")

buffer = SimulatedStockBuffer.objects.select_related("stock").order_by("captured_at").all()

price_variations = {}
for snapshot in buffer:
    if snapshot.stock.name not in price_variations:
        price_variations[snapshot.stock.name] = []
    price_variations[snapshot.stock.name].append(snapshot.price_snapshot)

# print(price_variations)
for stck_name in price_variations:
    price_snaps = price_variations[stck_name]
    price_array = np.array(price_snaps)
    max_price = max(price_array)
    normalized_snaps = price_array/max_price
    price_variations[stck_name] = normalized_snaps

    

stock_names = list(price_variations.keys())


# print(price_variations)
df_regret_comparisons = pd.DataFrame(price_variations)
df_regret_comparisons.iplot(
    xTitle='time_steps',
    yTitle='Normalized Prices'
)

In [10]:
last_9_objects = SimulatedStockBuffer.objects.filter(stock_id = 318).order_by('-captured_at')[:9].values('captured_at', 'price_snapshot', 'volume')
last_9_objects = list(reversed(last_9_objects))
print(last_9_objects)

[{'captured_at': datetime.datetime(1995, 3, 18, 21, 0, tzinfo=datetime.timezone.utc), 'price_snapshot': 7.647669717365825, 'volume': None}, {'captured_at': datetime.datetime(1995, 3, 18, 23, 30, tzinfo=datetime.timezone.utc), 'price_snapshot': 7.525568258004343, 'volume': None}, {'captured_at': datetime.datetime(1995, 3, 19, 2, 0, tzinfo=datetime.timezone.utc), 'price_snapshot': 7.654550160330033, 'volume': None}, {'captured_at': datetime.datetime(1995, 3, 19, 4, 30, tzinfo=datetime.timezone.utc), 'price_snapshot': 7.568721850159026, 'volume': None}, {'captured_at': datetime.datetime(1995, 3, 19, 7, 0, tzinfo=datetime.timezone.utc), 'price_snapshot': 7.454565801011557, 'volume': None}, {'captured_at': datetime.datetime(1995, 3, 19, 9, 30, tzinfo=datetime.timezone.utc), 'price_snapshot': 7.251004055947827, 'volume': None}, {'captured_at': datetime.datetime(1995, 3, 19, 12, 0, tzinfo=datetime.timezone.utc), 'price_snapshot': 7.378228483298571, 'volume': None}, {'captured_at': datetime.da

In [6]:
a_time_step = datetime(year=1995, month=1, day=7, hour=16, minute=00)
t_time_step = pytz.utc.localize(datetime.strptime(str(a_time_step), '%Y-%m-%d %H:%M:%S'))
a_next_time_step = t_time_step + relativedelta(hours=2, minutes=30)
ten_time_steps_ago = t_time_step - relativedelta(hours= 25)

# last_10_snapshots: list[SimulatedStockBuffer] = SimulatedStockBuffer.objects.filter(
#     Q(captured_at__gte=ten_time_steps_ago)
#     & Q(captured_at__lte=t_time_step)
#     & Q(stock_id = 318)
# ).order_by('-captured_at')
# last_10_snapshots = list(reversed(last_10_snapshots))
# blk_updts: list[SimulatedStockBuffer] = []

# for asnpsht in last_10_snapshots:
#     asnpsht.change = +1
#     asnpsht.volume = 1000
#     blk_updts.append(asnpsht)
# # blk_updts[7].change = 1
# blk_updts[8].change = -1
# blk_updts[9].change = -1

# blk_updts[9].captured_at
# # blk_updts[9].change = -1
# SimulatedStockBuffer.objects.bulk_update(blk_updts, ["change", "volume"])

stck_smltr = StockSimulator(a_time_step, a_next_time_step, is_coupled=True)
vols = stck_smltr.market_sentiment
print(json.dumps(vols, indent=3))

False
4117.69688204119 4074.27659355893
start
4074.27659355893
4117.69688204119

{
   "345": {
      "first_8_consecutives": 5,
      "latest_2_consecutives": 1,
      "latest_change": 0.010184361848924435,
      "index": 10,
      "price_x_volume_factor": 0.030112735814634917,
      "expected_volume_perc_change": 0.03198392401231392,
      "volume_x_price_factor": 0.33338335772283384,
      "volume_so_far": 1839.9297386839573,
      "vol_price_effect": 0.00406311377675934,
      "new_volume": 1861.2613800015936
   },
   "343": {
      "first_8_consecutives": -7,
      "latest_2_consecutives": -1,
      "latest_change": -0.029663671876132683,
      "index": 10,
      "price_x_volume_factor": 0.26868661472450533,
      "expected_volume_perc_change": 0,
      "volume_x_price_factor": 0.3793078121611786,
      "volume_so_far": 4074.27659355893,
      "vol_price_effect": 0.004338790879042177,
      "new_volume": 4117.69688204119
   },
   "342": {
      "first_8_consecutives": 7,
      "lat

In [7]:
np.zeros((10,6))

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])